In [1]:
import numpy as np
from collections import Counter
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from math import log2

# Function to calculate entropy of a dataset
def entropy(y):
    counts = Counter(y)
    total = len(y)
    return -sum((count/total) * log2(count/total) for count in counts.values())

# Function to calculate information gain
def information_gain(X, y, feature_index):
    total_entropy = entropy(y)
    values = np.unique(X[:, feature_index])
    weighted_entropy_sum = 0

    for value in values:
        subset_y = y[X[:, feature_index] == value]
        weighted_entropy_sum += (len(subset_y) / len(y)) * entropy(subset_y)

    return total_entropy - weighted_entropy_sum

# Function to split the dataset based on a feature and its value
def split_dataset(X, y, feature_index, value):
    indices = X[:, feature_index] == value
    return X[indices], y[indices]

# Class representing a Decision Tree
class DecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.tree = None

    def fit(self, X, y):
        self.tree = self._build_tree(X, y)

    def _build_tree(self, X, y, depth=0):
        # If all labels are the same, return a leaf node
        if len(set(y)) == 1:
            return {'label': y[0]}

        # If we reach the maximum depth, return a leaf node
        if self.max_depth is not None and depth >= self.max_depth:
            most_common = Counter(y).most_common(1)[0][0]
            return {'label': most_common}

        # Find the best feature to split on
        best_info_gain = -1
        best_split = None
        best_left_y = None
        best_right_y = None
        best_feature_index = None

        for feature_index in range(X.shape[1]):
            info_gain = information_gain(X, y, feature_index)
            if info_gain > best_info_gain:
                best_info_gain = info_gain
                best_feature_index = feature_index

                # Get the unique values for the best feature and split the dataset
                best_split = np.unique(X[:, best_feature_index])
                left_X, left_y = X[X[:, best_feature_index] == best_split[0]], y[X[:, best_feature_index] == best_split[0]]
                right_X, right_y = X[X[:, best_feature_index] == best_split[1]], y[X[:, best_feature_index] == best_split[1]]

                best_left_y = left_y
                best_right_y = right_y

        # Recursively build left and right subtrees
        left_subtree = self._build_tree(left_X, best_left_y, depth + 1)
        right_subtree = self._build_tree(right_X, best_right_y, depth + 1)

        return {
            'feature_index': best_feature_index,
            'split_value': best_split,
            'left': left_subtree,
            'right': right_subtree
        }

    def predict(self, X):
        return [self._predict_sample(sample, self.tree) for sample in X]

    def _predict_sample(self, sample, tree):
        if 'label' in tree:
            return tree['label']
        
        feature_value = sample[tree['feature_index']]
        
        if feature_value == tree['split_value'][0]:
            return self._predict_sample(sample, tree['left'])
        else:
            return self._predict_sample(sample, tree['right'])

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize and train the decision tree classifier
clf = DecisionTree(max_depth=3)
clf.fit(X_train, y_train)

# Make predictions on the test set
predictions = clf.predict(X_test)

# Calculate accuracy
accuracy = np.sum(predictions == y_test) / len(y_test)
print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 42.22%
